In [ ]:
import os

from srai.datasets import PortoTaxiDataset

%load_ext dotenv
%dotenv

In [ ]:
porto_taxi = PortoTaxiDataset()
hf_token = os.getenv("HF_TOKEN")

In [ ]:
type(porto_taxi.train_gdf), type(porto_taxi.test_gdf)

Get data using .load() method -> Default config (Travel time estimation)

In [ ]:
ds = porto_taxi.load(hf_token=hf_token)
ds.keys()

In [ ]:
type(porto_taxi.train_gdf), type(porto_taxi.test_gdf)

In [ ]:
ds["train"].head()

In [ ]:
ds["test"].head()

Getting h3 trajectories with target valeus

In [ ]:
porto_taxi.resolution

In [ ]:
train_h3, test_h3 = porto_taxi.get_h3_with_labels()

In [ ]:
train_h3

In [ ]:
test_h3

Get Human Mobility Classification (HMC) data

In [ ]:
ds = porto_taxi.load(hf_token=hf_token, version="HMC")

In [ ]:
ds["train"].head()

Creating your own train_test split based on trajectory duration (version TTE) or length version (HMC). Passing resolution parameter is neccessary for generation a trajectory in h3 style.

In [ ]:
porto_taxi_all = PortoTaxiDataset()

In [ ]:
ds = porto_taxi_all.load(hf_token=hf_token, version="all", resolution=9)
ds.keys()

In [ ]:
ds["train"].head()

In [ ]:
train, test = porto_taxi_all.train_test_split_bucket_trajectory(
    trajectory_id_column="trip_id", task="HMC", test_size=0.2, bucket_number=7
)

In [ ]:
len(train), len(test)